<h1 align='center'>Prediction on Readmission Rates</h1> <a id=100></a>

1. [Introduction](#1) <a id=18></a>
    - 1.1 [Data Dictionary](#2)
    - 1.2 [Task](#3)
2. [Preparation](#4)
    - 2.1 [Packages and Data](#5)
    - 2.2 [Preliminary Analysis and the Final Dataset](#6)
3. [Exploratory Data Analysis (EDA)](#7)
    - 3.1 [Univariate Analysis](#9)
    - 3.2 [Bivariate Analysis](#10)
4. [Data Preprocessing](#11)
    - 4.1 [Conclusions from the EDA](#12)
    - 4.2 [Packages](#13)
    - 4.3 [Making features model ready](#14)
5. [Modeling](#15)
    - 5.1 [Linear Classifiers](#16)
    - 5.2 [Tree Models](#17)

### Introduction <a id=1></a>
[back to top](#100)

The dataset represents 10 years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. It includes over 50 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria.

(1) It is an inpatient encounter (a hospital admission).

(2) It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.

(3) The length of stay was at least 1 day and at most 14 days.

(4) Laboratory tests were performed during the encounter.

(5) Medications were administered during the encounter.

The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc.

#### 1.1 Data Dictionary <a id=2></a>
`encounter_id` - Unique identifier of an encounter

`patient_nbr` - Unique identifier of a patient

`race` - Values: Caucasian, Asian, African American, Hispanic, and other

`gender` - Values: male, female, and unknown/invalid

`age` - Grouped in 10-year intervals: [0,10), [10,20), …, [90,100)

`weight` - Weight in pounds.

`admission_type_id` - Integer identifier corresponding to 9 distinct values, for example, emergency, urgent, elective, newborn, and not available	

`discharge_disposition_id`  - Integer identifier corresponding to 29 distinct values, for example, discharged to home, expired, and not available	

`admission_source_id` - Integer identifier corresponding to 21 distinct values, for example, physician referral, emergency room, and transfer from a hospital

`time_in_hospital` - Integer number of days between admission and discharge (1~14)

`payer_code` - Integer identifier corresponding to 23 distinct values, for example, Blue Cross/Blue Shield, Medicare, and self-pay	

`medical_specialty` - Integer identifier of a specialty of the admitting physician, corresponding to 84 distinct values, for example, cardiology, internal medicine, family/general practice, and surgeon

`num_lab_procedures` - Number of lab tests performed during the encounter	

`num_procedures` - Number of procedures (other than lab tests) performed during the encounter

`num_medications` - Number of distinct generic names administered during the encounter

`number_outpatient` - Number of outpatient visits of the patient in the year preceding the encounter	

`number_emergency` - Number of emergency visits of the patient in the year preceding the encounter	

`number_inpatient` - Number of inpatient visits of the patient in the year preceding the encounter	

`diag_1` - The primary diagnosis (coded as first three digits of ICD9); 848 distinct values	

`diag_2` - Secondary diagnosis (coded as first three digits of ICD9); 923 distinct values	

`diag_3` - Additional secondary diagnosis (coded as first three digits of ICD9); 954 distinct values	

`number_diagnoses` - Number of diagnoses entered to the system	

`max_glu_serum` - Indicates the range of the result or if the test was not taken. Values: “>200,” “>300,” “normal,” and “none” if not measured	

`A1Cresult` - Indicates the range of the result or if the test was not taken. Values: “>8” if the result was greater than 8%, “>7” if the result was greater than 7% but less than 8%, “normal” if the result was less than 7%, and “none” if not measured.	

23 features for medications -

`metformin`, 
`repaglinide`, `nateglinide`, `chlorpropamide`,
`glimepiride`, `acetohexamide`, `glipizide`, `glyburide`, `tolbutamide`, `pioglitazone`,
`rosiglitazone`, `acarbose`, `miglitol`, `troglitazone`, `tolazamide`, `examide`, `sitagliptin`, `insulin`,
`glyburide-metformin`, `glipizide-metformin`, `glimepiride-pioglitazone`,
`metformin-rosiglitazone`, `metformin-pioglitazone`, the feature indicates whether
the drug was prescribed or there was a change in the dosage. Values: “up” if the dosage
was increased during the encounter, “down” if the dosage was decreased, “steady” if the
dosage did not change, and “no” if the drug was not prescribed


`change` - Indicates if there was a change in diabetic medications (either dosage or generic name). Values: “change” and “no change”

`diabetesMed` - Indicates if there was any diabetic medication prescribed. Values: “yes” and “no”	

`readmitted` - Days to inpatient readmission. Values: “<30” if the patient was readmitted in less than 30 days, “>30” if the patient was readmitted in more than 30 days, and “No” for no record of readmission.	


### 2. Preparation <a id=4></a>
[back to top](#100)

#### 2.1 Import Packages and Load Data <a id=5></a>

In [66]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


In [2]:
# load datasets
df = pd.read_csv("/Users/kaiwenliu/Documents/Communication Requirement/diabete/dataset_diabetes/diabetic_data.csv")
name_map = pd.read_csv("/Users/kaiwenliu/Documents/Communication Requirement/diabete/dataset_diabetes/IDs_mapping.csv")


X = df.drop(columns=['readmitted']) #  features
y = df['readmitted'] #  target variable

# Convert the categorical target variable into numeric format
le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)

['<30' '>30' 'NO']


In [3]:
# # find the indices where the blank lines are located
# blank_line_indices = name_map.index[name_map.isnull().all(1)]

# # split the DataFrame into three separate DataFrames
# admission_type_id_map = name_map.iloc[:blank_line_indices[0]]
# discharge_disposition_id_map = name_map.iloc[blank_line_indices[0]+1:blank_line_indices[1]]
# dfadmission_source_id3 = name_map.iloc[blank_line_indices[1]+1:]

#### 2.2 Final dataset

##### 2.2.1 Drop dependent observations

In [4]:
# create an operation to drop certain ovservationsa
class DropObservations(BaseEstimator, TransformerMixin):
    def __init__(self):
        # id to remove for redundancy
        self.id_remove = []
        # id to remove for certain values in discharge_disposition_id        
        self.id_remove_dd = []
        self.rmPool = set([11,13,14,19,20,26])
     
    def update_drop_list(self,X):
        # We thus used only one encounter per patient; in particular, 
        # we considered only the first encounter for each patient as the primary admission and 
        # determined whether or not they were readmitted within 30 days. 

        patient_id_pool = set()
        for ind, val in X['patient_nbr'].items():
            if val in patient_id_pool:
                self.id_remove.append(ind)
            else:
                patient_id_pool.add(val)
        
        # Additionally, we removed all encounters that resulted in either discharge to a hospice 
        # (corresponeds to discharge_disposition_id=[13,14,19,20,26]) or 
        # patient death (correspondes to discharge_disposition_id=11), 
        # to avoid biasing our analysis.
        
        for i,v in X['discharge_disposition_id'].items():
            if v in self.rmPool:
                self.id_remove_dd.append(i)
                        
    def fit_transform(self, X, y):
        X_new = X.reset_index(drop=True)
        y_new = pd.Series(y)
        self.update_drop_list(X_new)
        # drop rows according to the ids and return
        ind_drop = self.id_remove+self.id_remove_dd
        return [X_new.drop(index=ind_drop), y_new.drop(index=ind_drop) ]

In [5]:
obj = DropObservations()
X,y = obj.fit_transform(X,y)
print(f'The shape of the current dataset is: {X.shape}')

The shape of the current dataset is: (69973, 49)


##### Check the distribution of the response variable

In [6]:
print(y.value_counts())
print(f'0: {le.classes_[0]},\n1: {le.classes_[1]},\n2: {le.classes_[2]}')

2    41474
1    22222
0     6277
dtype: int64
0: <30,
1: >30,
2: NO


#### 2.2.2 Split dataset into training and testing

In [7]:
# split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

# Next, use stratified sampling to further split the training set into training and validation sets
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=3202)

for train_index, val_index in strat_split.split(X_train, y_train):
    X_train_split, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_split, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

X_train = X_train_split
y_train = y_train_split

# reset index, make it starts from 0
X_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

X_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

X_val.reset_index(drop=True,inplace=True)
y_val.reset_index(drop=True,inplace=True)

In [8]:
print(f'The shape of the training dataset is {X_train.shape}')
print(f'The shape of the testing dataset is {X_test.shape}')
print(f'The shape of the validation dataset is {X_val.shape}')


The shape of the training dataset is (44782, 49)
The shape of the testing dataset is (13995, 49)
The shape of the validation dataset is (11196, 49)


In [15]:
y_train.value_counts()

2    26556
1    14190
0     4036
dtype: int64

### Impute gender

In [16]:
pd.Series(X_train['gender'].value_counts())

Female    23768
Male      21014
Name: gender, dtype: int64

In [17]:
imputer = SimpleImputer(missing_values = 'Unknown/Invalid',strategy='most_frequent')
imputer.fit(X_train[['gender']])
new_X = imputer.transform(X_train[['gender']])
X_train['gender'] = new_X

In [18]:
X_train['gender'].value_counts()

Female    23768
Male      21014
Name: gender, dtype: int64

##### 2.2.2 Group the categories from ICD-9 based on common attributes.

In [19]:
# icd9 codes and groups
# circulatory: 390–459, 785
# respiratory: 460–519, 786
# digestive: 520–579, 787
# diabetes: 250.xx
# injury: 800–999
# musculoskeletal: 710–739
# genitourinary: 580–629, 788
# neoplasms: 140–239
# other: other

def getICD9Group(val):
    category = ''
    try:
        num = int(float(val))
        if (num >=390 and num <=459 ) or num == 785:
            category = 'circulatory'
        elif (num >= 460 and num <= 519) or num == 786:
            category = 'respiratory'
        elif (num >= 520 and num <= 579) or num == 787:
            category = 'digestive'
        elif num==250:
            category = 'diabetes'
        elif num >= 800 and num <= 999:
            category = 'injury'
        elif num >= 710 and num <= 739:
            category = 'musculoskeletal'
        elif (num >= 580 and num <= 629) or num == 788:
            category = 'genitourinary'
        elif num >= 140 and num <= 239:
            category = 'neoplasms'
        else:
            category = 'other'
    except ValueError:
        category = 'other'
    return category

def getNewColumnByICD9Group(df, col, name):
    newCol = []
    for i, val in df[col].items():
        newCol.append(getICD9Group(val))
    return pd.Series(newCol, name = name)

# newCol_diag1 = getNewColumnByICD9Group(df_new, 'diag_1', 'diag_1_new')
# newCol_diag2 = getNewColumnByICD9Group(df_new, 'diag_2', 'diag_2_new')
# newCol_diag3 = getNewColumnByICD9Group(df_new, 'diag_3', 'diag_3_new')


# create an operation to group the categories from ICD-9 based on common attributes.
class GroupICD9IntoCatefories(BaseEstimator, TransformerMixin):
    def transform(self, X):
        newCol_diag1 = getNewColumnByICD9Group(X, 'diag_1', 'diag_1_new')
        newCol_diag2 = getNewColumnByICD9Group(X, 'diag_2', 'diag_2_new')
        newCol_diag3 = getNewColumnByICD9Group(X, 'diag_3', 'diag_3_new')
        df_new0 = X.reset_index()
        df_new1 = pd.concat([df_new0,newCol_diag1], axis=1)
        df_new2 = pd.concat([df_new1,newCol_diag2], axis=1)
        df_new3 = pd.concat([df_new2,newCol_diag3], axis=1)
        return df_new3



In [20]:
obj = GroupICD9IntoCatefories()
X_train1 = obj.transform(X_train)
print(f'The shape of the current dataset is: {X_train1.shape}')

The shape of the current dataset is: (44782, 53)


In [21]:
# check the result
X_train1['diag_3_new'].value_counts()

other              13466
circulatory        13410
diabetes            8106
respiratory         2986
genitourinary       2578
digestive           1722
injury               908
musculoskeletal      897
neoplasms            709
Name: diag_3_new, dtype: int64

##### 2.2.2 Drop unnecessary columns

In [22]:
# check the number of unqiue values of each variable
def check_unique_values(df):
    num_unique = pd.DataFrame(df.nunique())
    num_unique.columns = ['num_unique']
    return num_unique

check_unique_values(X_train1)

,num_unique
index,44782
encounter_id,44782
patient_nbr,44782
race,6
gender,2
age,10
weight,10
admission_type_id,8
discharge_disposition_id,21
admission_source_id,17


In [23]:
# remove weight and payer code since they have a high percentage of missing values
# (This is indicated in the paper[https://www.hindawi.com/journals/bmri/2014/781670/#introduction]
# drop 'examide', 'citoglipton' and 'glimepiride-pioglitazone' because they have only one category
# df = df_new3.drop(columns=['weight', 'payer_code', 'encounter_id', 'patient_nbr', 'diag_1', 'diag_2', 'diag_3', 'index', 'examide', 'citoglipton', 'glimepiride-pioglitazone'])

# check duplicates
# print(f"The number of duplicates: {df.duplicated().sum()}")


class DropFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features_drop = features
        
    def transform(self, X):
        return X.drop(columns=self.features_drop)


In [25]:
# since 'examide', 'citoglipton' and 'glimepiride-pioglitazone' have only one category, they can be dropped
drop_list=['weight', 'payer_code', 'encounter_id', 'patient_nbr', 'diag_1', 'diag_2', 
           'diag_3', 'examide','index', 'citoglipton', 'glimepiride-pioglitazone']
obj = DropFeatures(drop_list)
X_train2 = obj.transform(X_train1)
print(f'The shape of the current dataset is: {X_train2.shape}')

The shape of the current dataset is: (44782, 42)


In [26]:
# check
X_train2.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'diag_1_new', 'diag_2_new', 'diag_3_new'],
      dtype='object')

#### 2.4 Preliminary Analysis <a id=6></a>


##### 2.3.3 Group infrequent values into one a single category

In [27]:
# check the proportion of each category
def checkCategoryProp(se):
    counts = se.value_counts()
    prop = counts/counts.sum()
    return pd.DataFrame({
        'category': counts.index,
        'count': counts,
        'proportion': prop
    })


In [30]:
# print(check_unique_values(df_new3))
# checkCategoryProp(df_new3['admission_type_id'])
# checkCategoryProp(df_new3['discharge_disposition_id'])
# checkCategoryProp(df_new3['admission_source_id'])
table = checkCategoryProp(X_train2['medical_specialty'])
table[checkCategoryProp(X_train2['medical_specialty'])['proportion'] < 0.01]['category'].tolist()

['Pulmonology',
 'Psychiatry',
 'ObstetricsandGynecology',
 'Urology',
 'Surgery-Cardiovascular/Thoracic',
 'Surgery-Neuro',
 'Gastroenterology',
 'Surgery-Vascular',
 'Oncology',
 'Pediatrics',
 'PhysicalMedicineandRehabilitation',
 'Neurology',
 'Pediatrics-Endocrinology',
 'Hematology/Oncology',
 'Otolaryngology',
 'Surgery-Thoracic',
 'Endocrinology',
 'Surgery-Cardiovascular',
 'Podiatry',
 'Pediatrics-CriticalCare',
 'Psychology',
 'Gynecology',
 'Surgeon',
 'Radiology',
 'Osteopath',
 'Hematology',
 'Hospitalist',
 'Ophthalmology',
 'Surgery-Plastic',
 'InfectiousDiseases',
 'SurgicalSpecialty',
 'Anesthesiology-Pediatric',
 'Obstetrics',
 'Obsterics&Gynecology-GynecologicOnco',
 'Surgery-Colon&Rectal',
 'OutreachServices',
 'PhysicianNotFound',
 'Cardiology-Pediatric',
 'Pathology',
 'Pediatrics-Pulmonology',
 'Anesthesiology',
 'Rheumatology',
 'Pediatrics-EmergencyMedicine',
 'Psychiatry-Child/Adolescent',
 'Surgery-Maxillofacial',
 'Endocrinology-Metabolism',
 'Surgery-Pedia

In [31]:
# group infrequent values into one a single category
def group_infrequent_category(df, col_name, threshold, group_value):
    # change all the values that are below the threshold to the groupValue
    # return removed categories 
    table = checkCategoryProp(df[col_name])
    vals_to_group = table[(table['proportion'] < threshold)]['category'].tolist()
    df[col_name].replace(vals_to_group, group_value, inplace=True) 
    return vals_to_group

# create an operation to group infrequent values into one a single category
class GroupInfrequentCategories(BaseEstimator, TransformerMixin):
    
    def __init__(self, use_past_result=False, dict_cols_cats={}):
        self.use_past_result = use_past_result
        self.dict_cols_cats = dict_cols_cats
        
    def transform(self, X):
        if self.use_past_result:
            X['discharge_disposition_id'].replace(self.dict_cols_cats['discharge_disposition_id'], 0, inplace=True)
            X['admission_source_id'].replace(self.dict_cols_cats['admission_source_id'], 0, inplace=True)
            X['medical_specialty'].replace(self.dict_cols_cats['medical_specialty'], 'other', inplace=True)         
            
        else:
            dd_vals = group_infrequent_category(X,'discharge_disposition_id', 0.01, 0)
            ads_vals = group_infrequent_category(X, 'admission_source_id', 0.01, 0)
            ms_vals = group_infrequent_category(X, 'medical_specialty', 0.01, 'other')
#             store the results
            self.dict_cols_cats = {
                'discharge_disposition_id': dd_vals,
                'admission_source_id': ads_vals,
                'medical_specialty': ms_vals,
            }
            
        return X
    

In [32]:
obj = GroupInfrequentCategories()
obj.transform(X_train2)

# parameters to input for test data
dict_cols_cats = obj.dict_cols_cats

print(f'The shape of the current dataset is: {X_train2.shape}')
checkCategoryProp(X_train2['medical_specialty'])

The shape of the current dataset is: (44782, 42)


,category,count,proportion
?,?,21465,0.479322
InternalMedicine,InternalMedicine,6787,0.151556
other,other,3929,0.087736
Family/GeneralPractice,Family/GeneralPractice,3190,0.071234
Emergency/Trauma,Emergency/Trauma,2807,0.062681
Cardiology,Cardiology,2714,0.060605
Surgery-General,Surgery-General,1408,0.031441
Orthopedics,Orthopedics,729,0.016279
Orthopedics-Reconstructive,Orthopedics-Reconstructive,717,0.016011
Radiologist,Radiologist,529,0.011813


### 3. Exploratory Data Analysis (EDA) <a id=7></a>
[back to top](#100)

#### 3.1 Univariate Analysis <a id=9></a>

##### 3.1.1 Split features into categorical and numerical

In [33]:
X_train2.shape

(44782, 42)

In [34]:
categorical_features = ['race', 'gender', 'age', 'admission_type_id','discharge_disposition_id', 
                        'admission_source_id', 'medical_specialty', 'max_glu_serum', 'A1Cresult',
                        'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
                        'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
                        'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin',
                        'glyburide-metformin', 'glipizide-metformin', 'metformin-rosiglitazone',
                        'metformin-pioglitazone', 'change', 'diabetesMed', 'diag_1_new', 'diag_2_new',
                        'diag_3_new'
                       ]

numerical_features = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 
                      'number_outpatient', 'number_emergency','number_inpatient', 'number_diagnoses']

print(len(categorical_features))
print(len(numerical_features))


34
8


In [35]:
# Convert selected columns to categorical dtype
# for col in categorical_features:
#     [col] = df_new3[col].astype('category')
    

# create an operation to change type of certain columns
class ChangeColumnType(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features
        
    def transform(self, X):
        for col in self.categorical_features:
            X[col] = X[col].astype('category')
        return X
    

In [36]:
obj = ChangeColumnType(categorical_features)
obj.transform(X_train2)
print(X_train2.dtypes)

race                        category
gender                      category
age                         category
admission_type_id           category
discharge_disposition_id    category
admission_source_id         category
time_in_hospital               int64
medical_specialty           category
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
number_outpatient              int64
number_emergency               int64
number_inpatient               int64
number_diagnoses               int64
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
nateglinide                 category
chlorpropamide              category
glimepiride                 category
acetohexamide               category
glipizide                   category
glyburide                   category
tolbutamide                 category
pioglitazone                category
r

#### 3.1.2 Chi-Square test 

In [37]:
# def chi2_test(x, y, col):
#     # Compute the contingency table
#     contingency_table = pd.crosstab(x[col], y)
#     # Compute the chi-square test statistic and p-value
#     chi2, pval, dof, expected = chi2_contingency(contingency_table)
#     return [chi2, pval, dof, expected]

# chi2s = []
# pvals = []
# dofs = []
# for c in categorical_features:
#     chi2, pval, dof, _ = chi2_test(X_train2,y_train,c)
#     chi2s.append(chi2)
#     pvals.append(pval)
#     dofs.append(dof)


# significant = ['*' if x<0.05 else '' for x in pvals]

# chi2_table = pd.DataFrame({
#     'chi2': chi2s,
#     'pval': pvals,
#     'dof': dofs,
#     'significance': significant,
# })

# index_drop = [ind for ind, val in enumerate(pvals) if val>=0.05]


# create an operation to change type of certain columns
class drop_features_with_low_chi2_pval(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical_features, significance_level=0.1):
        self.index_drop = []
        self.significance_level = significance_level
        self.table = pd.DataFrame
        self.categorical_features=categorical_features
        
#     function of conducting chi-square test
    def chi2_test(cls, X, y, col):
        # Compute the contingency table
        contingency_table = pd.crosstab(X[col], y)
        # Compute the chi-square test statistic and p-value
        chi2, pval, dof, expected = chi2_contingency(contingency_table)
        return [chi2, pval, dof, expected]

    def plot_chi2_table(self, X, y):
        chi2s = []
        pvals = []
        dofs = []
        for c in self.categorical_features:
            chi2, pval, dof, _ = self.chi2_test(X,y,c)
            chi2s.append(chi2)
            pvals.append(pval)
            dofs.append(dof)

        significant = ['*' if x<self.significance_level else '' for x in pvals]

        chi2_table = pd.DataFrame({
            'chi2': chi2s,
            'pval': pvals,
            'dof': dofs,
            'significance': significant,
        })

        return chi2_table
    
    def fit(self, X, y):
        self.table = self.plot_chi2_table(X, y)
        pvals = self.table['pval']
        self.index_drop = [ind for ind, val in enumerate(pvals) if val>=self.significance_level]
        
    def transform(self, X):
        cols = [self.categorical_features[i] for i in self.index_drop]
        return X.drop(columns=cols)
    

In [38]:
# obj = drop_features_with_low_chi2_pval()
# obj.plot_chi2_table(X_train2,y_train, categorical_features)

In [39]:
obj = drop_features_with_low_chi2_pval(categorical_features)
obj.fit(X_train2, y_train)
X_train3 = obj.transform(X_train2)
print(X_train3.shape)
columns_dropped_due_to_chi2_test = [categorical_features[i] for i in obj.index_drop]

(44782, 29)


In [40]:
print(columns_dropped_due_to_chi2_test)

['nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glyburide', 'tolbutamide', 'miglitol', 'troglitazone', 'tolazamide', 'glyburide-metformin', 'glipizide-metformin', 'metformin-rosiglitazone', 'metformin-pioglitazone']


### 4. Preparing data for modeling

#### 5.1 Handle categorical features

In [41]:

# create an operation to get dummy varibles of categorical variables
class GetDummyVariable(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features
    
    def replace_illegal_name(self, X_train_dummies):
        # replace '[', ']' and '<' because they are invalud as column names in xgboost
        # replace '[' with '('; '<' with 'less than'; '>' with 'greater than'
        for s in X_train_dummies.columns:
            new_name = re.sub(r'\[', '(', s)
            X_train_dummies.rename(columns={s:new_name},inplace=True)

        for s in X_train_dummies.columns:
            new_name = re.sub(r'\<', 'less_than', s)
            X_train_dummies.rename(columns={s:new_name},inplace=True)

        for s in X_train_dummies.columns:
            new_name = re.sub(r'\>', 'greater_than', s)
            X_train_dummies.rename(columns={s:new_name},inplace=True)
        return X_train_dummies
    
    def transform(self, X):
        # get dummy variables
        X_train_dummies = pd.get_dummies(X, columns=self.categorical_features)
        return self.replace_illegal_name(X_train_dummies)


In [42]:
categorical_features_after_chi2 = X_train3.columns[X_train3.dtypes == 'category']
obj = GetDummyVariable(categorical_features_after_chi2)
X_train4 = obj.transform(X_train3)

In [43]:
print(X_train4.shape)

(44782, 127)


### 5. Modeling

#### 5.1 Random Forest

In [67]:
# standardize the data
# scaler = StandardScaler()

# Fit the scaler to the data and transform the data
# X_train_scaled = scaler.fit_transform(X_train4)

rfc = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rfc.fit(X_train_scaled, y_train)


# initialize GridSearchCV with cross-validation
grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=5)

# fit the grid search to the training data
grid_search.fit(X_train_scaled, y_train)

# print the best parameters found
print("Best parameters: ", grid_search.best_params_)


KeyboardInterrupt: 

In [ ]:
# pipeline for proprecessing
logist_preprocessing_pipeline = Pipeline([
    ('change ICD9 codes to categories', GroupICD9IntoCatefories()),
    ('drop features due to dependent rows', DropFeatures(drop_list)),
    ('group infrequent categories into one category', GroupInfrequentCategories(use_past_result=False, dict_cols_cats=dict_cols_cats)),
    ('change column type', ChangeColumnType(categorical_features)),
    ('drop features due to small pval in chi2 test', DropFeatures(columns_dropped_due_to_chi2_test)),
    ('get dummy variable for cateforical variables', GetDummyVariable(categorical_features_after_chi2)),
    ('scale', StandardScaler())
])

In [ ]:
# evaluate the best model on the training data
best_logreg = grid_search.best_estimator_
X_val_processed = logist_preprocessing_pipeline.fit_transform()
training_score = best_logreg.score(X_train4, y_train)
print("Training accuracy: {:.3f}".format(training_score))

#### 6.1 XGboost

##### 6.1.1 train

In [47]:
# initialize XGBoost classifier
clf = XGBClassifier(objective="multi:softprob", random_state=2022)

# define the parameter grid to search over
param_grid = {
    'n_estimators': [500],
    'max_depth': [3,5],
    'learning_rate': [0.1],
    'subsample': [0.5],
    'colsample_bytree': [0.2,0.5],
    'num_class': [3],
}

# initialize GridSearchCV with cross-validation
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)

# fit the grid search to the training data
grid_search.fit(X_train4, y_train)

# print the best parameters found
print("Best parameters: ", grid_search.best_params_)


Best parameters:  {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'num_class': 3, 'subsample': 0.5}


In [48]:
# evaluate the best model on the training data
best_clf = grid_search.best_estimator_
training_score = best_clf.score(X_train4, y_train)
print("Training accuracy: {:.3f}".format(training_score))

Training accuracy: 0.630


In [49]:
# pipeline for proprecessing
xgboost_preprocessing_pipeline = Pipeline([
    ('change ICD9 codes to categories', GroupICD9IntoCatefories()),
    ('drop features due to dependent rows', DropFeatures(drop_list)),
    ('group infrequent categories into one category', GroupInfrequentCategories(use_past_result=False, dict_cols_cats=dict_cols_cats)),
    ('change column type', ChangeColumnType(categorical_features)),
    ('drop features due to small pval in chi2 test', DropFeatures(columns_dropped_due_to_chi2_test)),
    ('get dummy variable for cateforical variables', GetDummyVariable(categorical_features_after_chi2))   
])

In [50]:
X_test_preprocessed = xgboost_preprocessing_pipeline.transform(X_test)
X_train_pre = xgboost_preprocessing_pipeline.transform(X_train)

In [51]:
X_train_pre.shape

(44782, 127)

In [52]:
X_test_preprocessed.shape

(13995, 127)

In [40]:
y_test.shape

(13995,)

##### 6.1.2 predict

In [53]:
# evaluate the best model on the test data
best_clf = grid_search.best_estimator_
test_score = best_clf.score(X_test_preprocessed, y_test)
print("Test accuracy: {:.3f}".format(test_score))

Test accuracy: 0.614


<h1 align='center'>Appendix</h1>


`admission_type_id` - admission type - \
    Emergency = 1, \
    Urgent = 2, \
    Elective = 3, \
    Newborn = 4, \
    Not Available = 5, \
    Null = 6, \
    Trauma Center = 7, \
    Not Mapped = 8 \

`discharge_disposition_id`  - Discharge disposition - \
Discharged to home = 1, \
Discharged/transferred to another short term hospital = 2,\
Discharged/transferred to SNF = 3, \
Discharged/transferred to ICF = 4, \
Discharged/transferred to another type of inpatient care institution = 5, \
Discharged/transferred to home with home health service = 6, \
Left AMA = 7, \
Discharged/transferred to home under care of Home IV provider = 8, \
Admitted as an inpatient to this hospital = 9, \
Neonate discharged to another hospital for neonatal aftercare = 10, \
Expired = 11, \
Still patient or expected to return for outpatient services = 12, \
Hospice / home = 13, \
Hospice / medical facility = 14, \
Discharged/transferred within this institution to Medicare approved swing bed = 15, \
Discharged/transferred/referred another institution for outpatient services = 16, \
Discharged/transferred/referred to this institution for outpatient services = 17, \
NULL = 18, \
Expired at home. Medicaid only, hospice. = 19, \
Expired in a medical facility. Medicaid only, hospice. = 20, \
Discharged/transferred to another rehab fac including rehab units of a hospital . = 22, \
Discharged/transferred to a long term care hospital. = 23, \
Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare. = 24, \
Not Mapped = 25, \
Unknown/Invalid = 26, \
Discharged/transferred to a federal health care facility. = 27, \
Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital = 28, \
Discharged/transferred to a Critical Access Hospital (CAH). = 29, \
Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere = 30, 

`admission_source_id` - admission source - \
Physician Referral = 1, \
Clinic Referral = 2, \
HMO Referral = 3, \
Transfer from a hospital = 4, \
 Transfer from a Skilled Nursing Facility (SNF) = 5, \
 Transfer from another health care facility = 6, \
 Emergency Room = 7, \
 Court/Law Enforcement = 8, \
 Not Available = 9, \
 Transfer from critial access hospital = 10, \
Normal Delivery = 11, \
 Premature Delivery = 12, \
 Sick Baby = 13, \
 Extramural Birth = 14, \
Not Available = 15, \
NULL = 17, \
 Transfer From Another Home Health Agency = 18, \
Readmission to Same Home Health Agency = 19, \
 Not Mapped = 20, \
Unknown/Invalid = 21, \
 Transfer from hospital inpt/same fac reslt in a sep claim = 22, \
 Born inside this hospital = 23, \
 Born outside this hospital = 24, \
 Transfer from Ambulatory Surgery Center = 25, \
Transfer from Hospice = 26,
 